In [2]:
# Fix numpy/pandas compatibility issue
%pip install --upgrade numpy pandas matplotlib seaborn plotly --quiet

Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\users\\vtc\\appdata\\local\\packages\\pythonsoftwarefoundation.python.3.11_qbz5n2kfra8p0\\localcache\\local-packages\\python311\\site-packages\\matplotlib\\backends\\_backend_agg.cp311-win_amd64.pyd'
Check the permissions.


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\VTC\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## I. Setup & Data Loading

In [5]:
# Import libraries
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from pathlib import Path

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
%matplotlib inline

# Set styles
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
pd.set_option('display.max_columns', None)

print("Libraries imported successfully")

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
# Load all feature datasets
BASE_DIR = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
FEATURES_DIR = BASE_DIR / "data/features"

# Load corrosion features
corrosion_df = pd.read_csv(FEATURES_DIR / "corrosion_features.csv")
print(f"Pipeline Corrosion: {corrosion_df.shape}")

# Load bearing features
bearing_df = pd.read_csv(FEATURES_DIR / "bearing_features.csv")
print(f"Bearing: {bearing_df.shape}")

# Load pump features
pump_df = pd.read_csv(FEATURES_DIR / "pump_features.csv")
pump_df['timestamp'] = pd.to_datetime(pump_df['timestamp'])
print(f"Pump: {pump_df.shape}")

# Load turbine features
turbine_df = pd.read_csv(FEATURES_DIR / "turbine_features.csv")
turbine_df['timestamp'] = pd.to_datetime(turbine_df['timestamp'])
print(f"Turbine: {turbine_df.shape}")

print("\nAll datasets loaded successfully!")

### 1.1 Equipment Overview Summary

In [ ]:
# Create summary dataframe
equipment_summary = pd.DataFrame({
    'Equipment Type': ['Pipeline Corrosion', 'Bearing', 'Pump', 'Turbine'],
    'Total Records': [
        len(corrosion_df),
        len(bearing_df),
        len(pump_df),
        len(turbine_df)
    ],
    'Unique Equipment': [
        corrosion_df['equipment_id'].nunique(),
        bearing_df['equipment_id'].nunique(),
        pump_df['equipment_id'].nunique(),
        turbine_df['equipment_id'].nunique()
    ],
    'Mean Health Index': [
        1 - corrosion_df['risk_score'].mean()/100,  # Invert risk to health
        bearing_df['health_index'].mean(),
        pump_df['health_index'].mean(),
        turbine_df['health_index'].mean()
    ],
    'Anomaly Rate (%)': [
        (corrosion_df['condition'] == 'Critical').sum() / len(corrosion_df) * 100,
        bearing_df['is_anomaly'].sum() / len(bearing_df) * 100,
        pump_df['is_anomaly'].sum() / len(pump_df) * 100,
        turbine_df['is_anomaly'].sum() / len(turbine_df) * 100
    ]
})

print("\n=" * 80)
print(" " * 25 + "EQUIPMENT OVERVIEW")
print("=" * 80)
display(equipment_summary)

# Bar chart
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Records
axes[0].bar(equipment_summary['Equipment Type'], equipment_summary['Total Records'], 
            color=['#e74c3c', '#3498db', '#2ecc71', '#f39c12'])
axes[0].set_title('Total Records by Equipment', fontweight='bold')
axes[0].set_ylabel('Records')
axes[0].tick_params(axis='x', rotation=45)

# Health Index
axes[1].bar(equipment_summary['Equipment Type'], equipment_summary['Mean Health Index'],
            color=['#e74c3c', '#3498db', '#2ecc71', '#f39c12'])
axes[1].set_title('Mean Health Index', fontweight='bold')
axes[1].set_ylabel('Health Index (0-1)')
axes[1].set_ylim([0, 1])
axes[1].axhline(y=0.5, color='red', linestyle='--', linewidth=1, label='Critical threshold')
axes[1].tick_params(axis='x', rotation=45)
axes[1].legend()

# Anomaly Rate
axes[2].bar(equipment_summary['Equipment Type'], equipment_summary['Anomaly Rate (%)'],
            color=['#e74c3c', '#3498db', '#2ecc71', '#f39c12'])
axes[2].set_title('Anomaly Detection Rate', fontweight='bold')
axes[2].set_ylabel('Anomaly Rate (%)')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## II. Pipeline Corrosion Analysis

In [ ]:
print("=" * 70)
print(" " * 20 + "PIPELINE CORROSION EDA")
print("=" * 70)

print(f"\nDataset: {corrosion_df.shape}")
print(f"Equipment: {corrosion_df['equipment_id'].nunique()} pipeline segments")
print(f"\nKey Features:")
print(f"  - Corrosion rate: {corrosion_df['corrosion_rate_mm_year'].mean():.3f} mm/year")
print(f"  - Remaining life: {corrosion_df['remaining_life_years'].mean():.1f} years")
print(f"  - Risk score: {corrosion_df['risk_score'].mean():.1f}/100")
print(f"  - Safety margin: {corrosion_df['safety_margin_percent'].mean():.1f}%")

# Condition distribution
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Condition pie chart
condition_counts = corrosion_df['condition'].value_counts()
colors_cond = ['#e74c3c', '#f39c12', '#2ecc71']
axes[0].pie(condition_counts.values, labels=condition_counts.index, autopct='%1.1f%%',
           colors=colors_cond, startangle=90)
axes[0].set_title('Pipeline Condition Distribution', fontweight='bold')

# Risk score distribution
axes[1].hist(corrosion_df['risk_score'], bins=30, color='coral', edgecolor='black', alpha=0.7)
axes[1].set_title('Risk Score Distribution', fontweight='bold')
axes[1].set_xlabel('Risk Score (0-100)')
axes[1].set_ylabel('Frequency')
axes[1].axvline(corrosion_df['risk_score'].mean(), color='red', linestyle='--', 
               label=f'Mean: {corrosion_df["risk_score"].mean():.1f}')
axes[1].legend()

# Remaining life vs risk
for condition in corrosion_df['condition'].unique():
    subset = corrosion_df[corrosion_df['condition'] == condition]
    color = {'Critical': '#e74c3c', 'Moderate': '#f39c12', 'Normal': '#2ecc71'}.get(condition, 'gray')
    axes[2].scatter(subset['risk_score'], subset['remaining_life_years'], 
                   label=condition, alpha=0.6, s=50, color=color)
axes[2].set_xlabel('Risk Score')
axes[2].set_ylabel('Remaining Life (years)')
axes[2].set_title('Risk vs Remaining Life', fontweight='bold')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## III. Bearing Analysis

In [ ]:
print("=" * 70)
print(" " * 25 + "BEARING EDA")
print("=" * 70)

print(f"\nDataset: {bearing_df.shape}")
print(f"Equipment: {bearing_df['equipment_id'].nunique()} bearings")
print(f"\nKey Features:")
print(f"  - Mean RMS: {bearing_df['rms'].mean():.3f}")
print(f"  - Mean Health Index: {bearing_df['health_index'].mean():.3f}")
print(f"  - Anomalies: {bearing_df['is_anomaly'].sum()} ({bearing_df['is_anomaly'].sum()/len(bearing_df)*100:.1f}%)")

# Bearing health analysis
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Health index distribution
axes[0, 0].hist(bearing_df['health_index'], bins=30, color='steelblue', edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Health Index Distribution', fontweight='bold')
axes[0, 0].set_xlabel('Health Index')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].axvline(bearing_df['health_index'].mean(), color='red', linestyle='--', 
                  label=f'Mean: {bearing_df["health_index"].mean():.3f}')
axes[0, 0].legend()

# RMS by equipment
equipment_rms = bearing_df.groupby('equipment_id')['rms'].mean().sort_values(ascending=False)
axes[0, 1].barh(range(len(equipment_rms)), equipment_rms.values, color='teal')
axes[0, 1].set_yticks(range(len(equipment_rms)))
axes[0, 1].set_yticklabels(equipment_rms.index)
axes[0, 1].set_title('Mean RMS by Bearing', fontweight='bold')
axes[0, 1].set_xlabel('RMS')

# Health index by equipment
equipment_health = bearing_df.groupby('equipment_id')['health_index'].mean().sort_values()
colors_health = ['#e74c3c' if h < 0.5 else '#f39c12' if h < 0.7 else '#2ecc71' for h in equipment_health.values]
axes[1, 0].barh(range(len(equipment_health)), equipment_health.values, color=colors_health)
axes[1, 0].set_yticks(range(len(equipment_health)))
axes[1, 0].set_yticklabels(equipment_health.index)
axes[1, 0].set_title('Mean Health Index by Bearing', fontweight='bold')
axes[1, 0].set_xlabel('Health Index')
axes[1, 0].axvline(x=0.5, color='red', linestyle='--', linewidth=1)

# Anomaly rate by equipment
anomaly_rate = bearing_df.groupby('equipment_id')['is_anomaly'].mean() * 100
anomaly_rate = anomaly_rate.sort_values(ascending=False)
axes[1, 1].bar(range(len(anomaly_rate)), anomaly_rate.values, color='coral')
axes[1, 1].set_xticks(range(len(anomaly_rate)))
axes[1, 1].set_xticklabels(anomaly_rate.index, rotation=45, ha='right')
axes[1, 1].set_title('Anomaly Rate by Bearing', fontweight='bold')
axes[1, 1].set_ylabel('Anomaly Rate (%)')

plt.tight_layout()
plt.show()

## IV. Pump Analysis

In [ ]:
print("=" * 70)
print(" " * 27 + "PUMP EDA")
print("=" * 70)

print(f"\nDataset: {pump_df.shape}")
print(f"Equipment: {pump_df['equipment_id'].nunique()} pumps")
print(f"Date range: {pump_df['timestamp'].min()} to {pump_df['timestamp'].max()}")
print(f"\nKey Features:")
print(f"  - Mean Efficiency: {pump_df['efficiency_normalized'].mean():.3f}")
print(f"  - Mean Health Index: {pump_df['health_index'].mean():.3f}")
print(f"  - Mean RUL: {pump_df['rul_days'].mean():.0f} days")
print(f"  - Anomalies: {pump_df['is_anomaly'].sum()} ({pump_df['is_anomaly'].sum()/len(pump_df)*100:.1f}%)")

# Pump performance analysis
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Health index over time
for pump_id in pump_df['equipment_id'].unique():
    pump_data = pump_df[pump_df['equipment_id'] == pump_id].sort_values('timestamp')
    axes[0, 0].plot(pump_data['timestamp'], pump_data['health_index'], label=pump_id, linewidth=2)
axes[0, 0].set_title('Health Index Over Time', fontweight='bold')
axes[0, 0].set_xlabel('Timestamp')
axes[0, 0].set_ylabel('Health Index')
axes[0, 0].axhline(y=0.4, color='red', linestyle='--', linewidth=1, label='Critical threshold')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Efficiency vs Health
for pump_id in pump_df['equipment_id'].unique():
    pump_data = pump_df[pump_df['equipment_id'] == pump_id]
    axes[0, 1].scatter(pump_data['efficiency_normalized'], pump_data['health_index'], 
                      label=pump_id, alpha=0.5, s=20)
axes[0, 1].set_xlabel('Efficiency Normalized')
axes[0, 1].set_ylabel('Health Index')
axes[0, 1].set_title('Efficiency vs Health Index', fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# RUL distribution
for pump_id in pump_df['equipment_id'].unique():
    pump_data = pump_df[pump_df['equipment_id'] == pump_id]
    axes[1, 0].hist(pump_data['rul_days'], bins=30, alpha=0.6, label=pump_id, edgecolor='black')
axes[1, 0].set_xlabel('RUL (days)')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('RUL Distribution', fontweight='bold')
axes[1, 0].legend()

# Temperature over time
for pump_id in pump_df['equipment_id'].unique():
    pump_data = pump_df[pump_df['equipment_id'] == pump_id].sort_values('timestamp')
    axes[1, 1].plot(pump_data['timestamp'], pump_data['valueTEMP'], label=pump_id, linewidth=2)
axes[1, 1].set_title('Temperature Over Time', fontweight='bold')
axes[1, 1].set_xlabel('Timestamp')
axes[1, 1].set_ylabel('Temperature (°C)')
axes[1, 1].axhline(y=80, color='red', linestyle='--', linewidth=1, label='High temp threshold')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## V. Turbine/Engine Analysis

In [ ]:
print("=" * 70)
print(" " * 23 + "TURBINE/ENGINE EDA")
print("=" * 70)

print(f"\nDataset: {turbine_df.shape}")
print(f"Equipment: {turbine_df['equipment_id'].nunique()} turbofan engines")
print(f"Cycle range: {turbine_df['time_cycles'].min()} to {turbine_df['time_cycles'].max()}")
print(f"\nKey Features:")
print(f"  - Mean Health Index: {turbine_df['health_index'].mean():.3f}")
print(f"  - Mean RUL: {turbine_df['rul_actual'].mean():.1f} cycles")
print(f"  - Mean Temperature (T30): {turbine_df['sensor_4'].mean():.1f}")
print(f"  - Mean Pressure Ratio: {turbine_df['sensor_13'].mean():.2f}")
print(f"  - Anomalies: {turbine_df['is_anomaly'].sum()} ({turbine_df['is_anomaly'].sum()/len(turbine_df)*100:.1f}%)")

# Turbine degradation analysis
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Health index distribution by dataset
for dataset in turbine_df['dataset'].unique():
    data = turbine_df[turbine_df['dataset'] == dataset]
    axes[0, 0].hist(data['health_index'], bins=30, alpha=0.6, label=dataset, edgecolor='black')
axes[0, 0].set_xlabel('Health Index')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Health Index Distribution (Train vs Test)', fontweight='bold')
axes[0, 0].legend()

# RUL distribution
axes[0, 1].hist(turbine_df['rul_actual'], bins=50, color='steelblue', edgecolor='black', alpha=0.7)
axes[0, 1].set_xlabel('RUL (cycles)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('RUL Distribution', fontweight='bold')
axes[0, 1].axvline(turbine_df['rul_actual'].mean(), color='red', linestyle='--', 
                  label=f'Mean: {turbine_df["rul_actual"].mean():.1f}')
axes[0, 1].legend()

# Temperature gradient vs health
sample_turbines = turbine_df['equipment_id'].unique()[:5]  # Sample 5 turbines
for turbine_id in sample_turbines:
    turbine_data = turbine_df[turbine_df['equipment_id'] == turbine_id].sort_values('time_cycles')
    axes[1, 0].plot(turbine_data['time_cycles'], turbine_data['temp_gradient'], alpha=0.7, linewidth=1)
axes[1, 0].set_xlabel('Time Cycles')
axes[1, 0].set_ylabel('Temperature Gradient (T30-T2)')
axes[1, 0].set_title('Temperature Gradient Over Time (5 Sample Turbines)', fontweight='bold')
axes[1, 0].grid(alpha=0.3)

# Health vs RUL scatter
sample_data = turbine_df.sample(n=min(1000, len(turbine_df)))  # Sample for performance
scatter = axes[1, 1].scatter(sample_data['rul_actual'], sample_data['health_index'], 
                            c=sample_data['time_cycles'], cmap='viridis', alpha=0.6, s=20)
axes[1, 1].set_xlabel('RUL (cycles)')
axes[1, 1].set_ylabel('Health Index')
axes[1, 1].set_title('Health vs RUL (colored by cycle)', fontweight='bold')
plt.colorbar(scatter, ax=axes[1, 1], label='Time Cycles')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## VI. Cross-Equipment Comparison

In [ ]:
print("=" * 70)
print(" " * 20 + "CROSS-EQUIPMENT ANALYSIS")
print("=" * 70)

# Prepare data for comparison
comparison_data = []

# Corrosion
for _, row in corrosion_df.iterrows():
    comparison_data.append({
        'equipment_type': 'Pipeline',
        'health_index': 1 - row['risk_score']/100,
        'is_critical': row['condition'] == 'Critical'
    })

# Bearing
for _, row in bearing_df.iterrows():
    comparison_data.append({
        'equipment_type': 'Bearing',
        'health_index': row['health_index'],
        'is_critical': row['health_index'] < 0.4
    })

# Pump
for _, row in pump_df.iterrows():
    comparison_data.append({
        'equipment_type': 'Pump',
        'health_index': row['health_index'],
        'is_critical': row['health_index'] < 0.4
    })

# Turbine (sample to reduce size)
turbine_sample = turbine_df.sample(n=min(5000, len(turbine_df)))
for _, row in turbine_sample.iterrows():
    comparison_data.append({
        'equipment_type': 'Turbine',
        'health_index': row['health_index'],
        'is_critical': row['health_index'] < 0.4
    })

comparison_df = pd.DataFrame(comparison_data)

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Health index boxplot
equipment_types = ['Pipeline', 'Bearing', 'Pump', 'Turbine']
health_data = [comparison_df[comparison_df['equipment_type'] == et]['health_index'].values 
               for et in equipment_types]
bp = axes[0].boxplot(health_data, labels=equipment_types, patch_artist=True)
colors = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
axes[0].set_title('Health Index Distribution by Equipment', fontweight='bold')
axes[0].set_ylabel('Health Index')
axes[0].axhline(y=0.4, color='red', linestyle='--', linewidth=1, label='Critical threshold')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Critical equipment count
critical_counts = comparison_df.groupby('equipment_type')['is_critical'].sum()
axes[1].bar(equipment_types, [critical_counts.get(et, 0) for et in equipment_types], color=colors)
axes[1].set_title('Critical Equipment Count', fontweight='bold')
axes[1].set_ylabel('Count')
axes[1].tick_params(axis='x', rotation=45)

# Critical rate
critical_rate = comparison_df.groupby('equipment_type')['is_critical'].mean() * 100
axes[2].bar(equipment_types, [critical_rate.get(et, 0) for et in equipment_types], color=colors)
axes[2].set_title('Critical Rate by Equipment Type', fontweight='bold')
axes[2].set_ylabel('Critical Rate (%)')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("\nCritical Equipment Summary:")
print(comparison_df.groupby('equipment_type')['is_critical'].agg(['sum', 'mean']))

## VII. Key Findings & Recommendations

In [ ]:
print("=" * 80)
print(" " * 25 + "KEY FINDINGS")
print("=" * 80)

print("\n1. PIPELINE CORROSION:")
print(f"   - {(corrosion_df['condition']=='Critical').sum()} critical segments requiring immediate attention")
print(f"   - Mean remaining life: {corrosion_df['remaining_life_years'].mean():.1f} years")
print(f"   - Highest risk: {corrosion_df.nlargest(1, 'risk_score')['equipment_id'].values[0]}")

print("\n2. BEARING:")
print(f"   - {(bearing_df['health_index'] < 0.5).sum()} bearings below health threshold")
print(f"   - {bearing_df['is_anomaly'].sum()} anomalies detected across {bearing_df['equipment_id'].nunique()} bearings")
worst_bearing = bearing_df.groupby('equipment_id')['health_index'].mean().idxmin()
print(f"   - Worst performing: {worst_bearing}")

print("\n3. PUMP:")
print(f"   - {(pump_df['health_index'] < 0.4).sum()} critical pump records")
print(f"   - Mean efficiency: {pump_df['efficiency_normalized'].mean():.3f}")
print(f"   - Average RUL: {pump_df['rul_days'].mean():.0f} days (~{pump_df['rul_days'].mean()/365:.1f} years)")

print("\n4. TURBINE:")
print(f"   - {turbine_df['equipment_id'].nunique()} turbofan engines monitored")
print(f"   - Mean RUL: {turbine_df['rul_actual'].mean():.1f} cycles")
print(f"   - {turbine_df['is_anomaly'].sum()} anomalies detected ({turbine_df['is_anomaly'].sum()/len(turbine_df)*100:.2f}%)")

print("\n" + "=" * 80)
print(" " * 25 + "RECOMMENDATIONS")
print("=" * 80)

print("\n1. IMMEDIATE ACTIONS:")
print("   - Inspect critical pipeline segments with risk_score > 70")
print("   - Replace bearings with health_index < 0.4")
print("   - Schedule pump maintenance for units with RUL < 180 days")
print("   - Monitor turbines with frequent anomalies")

print("\n2. PREDICTIVE MODELING:")
print("   - Pipeline: Classification model (Critical/Moderate/Normal)")
print("   - Bearing: Binary classification (Healthy/Faulty)")
print("   - Pump: Regression for RUL prediction")
print("   - Turbine: RUL regression using LSTM/LightGBM")

print("\n3. MONITORING PRIORITIES:")
priority_order = equipment_summary.sort_values('Anomaly Rate (%)', ascending=False)
print("   Equipment priority (by anomaly rate):")
for idx, row in priority_order.iterrows():
    print(f"   {idx+1}. {row['Equipment Type']}: {row['Anomaly Rate (%)']:.1f}% anomaly rate")

print("\n4. NEXT STEPS:")
print("   - Run equipment-specific modeling notebooks:")
print("     * Pipeline_Corrosion_Modeling.ipynb (LightGBM classification)")
print("     * Bearing_Modeling.ipynb (if needed for retraining)")
print("     * Pump_Modeling.ipynb (if needed for RUL regression)")
print("     * Turbine_RUL_Modeling.ipynb (LSTM/LightGBM regression)")
print("   - Integrate predictions into real-time dashboard")
print("   - Set up automated alerting for critical equipment")

print("\n" + "=" * 80)
print("EDA COMPLETE")
print("=" * 80)